# Data Preprocessing

In [1]:
!pip install polars

import pandas as pd
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 31.8 MB/s eta 0:00:00


In [2]:
train = pd.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
test = pd.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

In [3]:
def map_column(df: pd.DataFrame, col_name: str):
    
    aid_sorted = sorted(list(df[col_name].unique()))
    mapping = {k: i + 2 for i, k in enumerate(aid_sorted)}
    inverse_mapping = {v: k for k, v in mapping.items()}

    df[col_name] = df[col_name].map(mapping)

    return df, mapping, inverse_mapping




train, mapping, inverse_mapping = map_column(train, 'aid')
test['aid'] = test['aid'].map(mapping)

In [4]:
train.loc[len(train.index)]  = [19000000, 0,0,0]

In [5]:
train.session = train.session.astype('int32')
train.ts = train.ts.astype('int32')
train.type =train.type.astype('uint8')

In [6]:
train = pl.DataFrame(train)
test = pl.DataFrame(test)



sentences_df = pl.concat([train, test]).groupby('session').agg(
    pl.col('aid').alias('sentence')
)

sentences = sentences_df['sentence'].to_list()

# Training a word2vec model

In [7]:
%%time

w2vec = Word2Vec(sentences=sentences, vector_size=32, min_count=1, workers=4)
w2vec.save("/kaggle/working/word2vec.model")

CPU times: user 1h 28min 29s, sys: 21.6 s, total: 1h 28min 51s
Wall time: 29min 1s
